In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv('data/properatti.csv')
data.head(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


In [3]:
''' Vemos las columnas '''

data.loc[10,:]


Unnamed: 0                                                                   10
operation                                                                  sell
property_type                                                             house
place_name                                                              Córdoba
place_with_parent_names                             |Argentina|Córdoba|Córdoba|
country_name                                                          Argentina
state_name                                                              Córdoba
geonames_id                                                         3.86026e+06
lat-lon                                                 -31.4200833,-64.1887761
lat                                                                    -31.4201
lon                                                                    -64.1888
price                                                                     70000
currency                                

In [4]:
''' vemos los nulos '''

data['price_aprox_usd'].isna().sum()


20410

In [5]:
''' Exploramos las columnas de los precios para ver si hay datos no nulos'''

data.loc[data['price_aprox_usd'].isna()].loc[:,['price','price_aprox_local_currency','price_usd_per_m2','price_per_m2']]




,price,price_aprox_local_currency,price_usd_per_m2,price_per_m2
5,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN
41,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,NaN
67,NaN,NaN,NaN,NaN
...,...,...,...,...
121210,NaN,NaN,NaN,NaN
121211,NaN,NaN,NaN,NaN
121212,NaN,NaN,NaN,NaN
121213,NaN,NaN,NaN,NaN


In [7]:
''' quitar los nulos precios '''

data = data.drop(data[data['price_aprox_usd'].isna()].index)
cant_nulls = data.isnull().sum()
print(cant_nulls)



Unnamed: 0                        0
operation                         0
property_type                     0
place_name                       23
place_with_parent_names           0
country_name                      0
state_name                        0
geonames_id                   16952
lat-lon                       42262
lat                           42262
lon                           42262
price                             0
currency                          1
price_aprox_local_currency        0
price_aprox_usd                   0
surface_total_in_m2           31944
surface_covered_in_m2         13150
price_usd_per_m2              32193
price_per_m2                  13152
floor                         93894
rooms                         61231
expenses                      87564
properati_url                     0
description                       2
title                             0
image_thumbnail                2250
dtype: int64


In [8]:
''' completar los places names '''
data.loc[data.place_name.isnull(),'place_name'] = data.loc[data.place_name.isnull()].place_with_parent_names.str.split('|', expand=True)[3]
cant_nulls = data.isnull().sum()
print(cant_nulls)


Unnamed: 0                        0
operation                         0
property_type                     0
place_name                        0
place_with_parent_names           0
country_name                      0
state_name                        0
geonames_id                   16952
lat-lon                       42262
lat                           42262
lon                           42262
price                             0
currency                          1
price_aprox_local_currency        0
price_aprox_usd                   0
surface_total_in_m2           31944
surface_covered_in_m2         13150
price_usd_per_m2              32193
price_per_m2                  13152
floor                         93894
rooms                         61231
expenses                      87564
properati_url                     0
description                       2
title                             0
image_thumbnail                2250
dtype: int64


In [9]:
''' Leo los ambientes para completar la info'''

import re

patron_ambientes = re.compile("(?P<Ambientes>\d+)(?P<tipo>amb|\samb)",flags = re.IGNORECASE)
patron_habitaciones = re.compile("(?P<Habitaciones>\d+)(?P<tipo>hab|\shab|dorm|\sdorm)",flags = re.IGNORECASE)

def extractRooms(row):
    ambientes_match = None
    if ( row['description'] and isinstance(row['description'],str) ):
        ambientes_match = patron_ambientes.search(row['description'])
    if ((ambientes_match is None) and row['title'] and isinstance(row['title'],str)):
        ambientes_match = patron_ambientes.search(row['title'])
    if (ambientes_match is not None):
        return int(ambientes_match.groups()[0]) 
    else: 
        if (row['description'] and isinstance(row['description'],str)):
            ambientes_match = patron_habitaciones.search(row['description'])
        if ((ambientes_match is None) and row['title'] and isinstance(row['title'],str)):
            ambientes_match = patron_habitaciones.search(row['title'])
        if (ambientes_match is not None):
            return int(ambientes_match.groups()[0])+1
        return None
        
        
data.loc[data.rooms.isnull(),'rooms'] = data.loc[data.rooms.isnull()].apply(extractRooms,axis=1)


In [ ]:
cant_nulls = data.isnull().sum()
print(cant_nulls)

In [ ]:

patron_meters = re.compile("(?P<Metros>\d+)(?P<tipo>metro|\smetro|mts|\smts|m2|\sm2)",flags = re.IGNORECASE)


def extractMeters(row):
    meters_match = None
    if ( row['description'] and isinstance(row['description'],str) ):
        meters_match = patron_meters.search(row['description'])
    if ((patron_meters is None) and row['title'] and isinstance(row['title'],str)):
        meters_match = patron_meters.search(row['title'])
    if (meters_match is not None):
        return meters_match.groups()[0]
    return None

data.loc[data.surface_total_in_m2.isnull(),'surface_total_in_m2'] = data.loc[data.surface_total_in_m2.isnull()].apply(extractMeters,axis=1)

data.loc[data.surface_total_in_m2.isnull() & data.surface_covered_in_m2.notnull(),'surface_total_in_m2'] = data.loc[data.surface_total_in_m2.isnull() & data.surface_covered_in_m2.notnull()].apply(lambda x: x['surface_covered_in_m2'],axis=1)




In [10]:
cant_nulls = data.isnull().sum()
print(cant_nulls)

Unnamed: 0                        0
operation                         0
property_type                     0
place_name                        0
place_with_parent_names           0
country_name                      0
state_name                        0
geonames_id                   16952
lat-lon                       42262
lat                           42262
lon                           42262
price                             0
currency                          1
price_aprox_local_currency        0
price_aprox_usd                   0
surface_total_in_m2           31944
surface_covered_in_m2         13150
price_usd_per_m2              32193
price_per_m2                  13152
floor                         93894
rooms                         24521
expenses                      87564
properati_url                     0
description                       2
title                             0
image_thumbnail                2250
dtype: int64


In [11]:
pd.options.display.float_format = '{:,.2f}'.format

data.loc[data.surface_total_in_m2.isnull(),'surface_total_in_m2'] = data.loc[data.surface_total_in_m2.isnull()].surface_total_in_m2.fillna(value=np.nan)
data.loc[:,'surface_total_in_m2'] = data.loc[:,'surface_total_in_m2'].astype("float")



In [38]:
data.loc[(data.property_type == 'apartment') & (data.floor.isnull())]


,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,"3,432,039.00","-34.9038831,-57.9643295",-34.90,...,nan,nan,nan,nan,3.00,nan,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,"3,430,787.00","-34.6522615,-58.5229825",-34.65,...,55.00,"1,309.09","1,309.09",nan,2.00,nan,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,"3,435,548.00","-38.0026256,-57.5494468",-38.00,...,35.00,"1,828.57","1,828.57",nan,2.00,nan,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...
7,7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,"3,436,077.00","-34.5598729,-58.443362",-34.56,...,40.00,"3,066.67","3,450.00",nan,nan,nan,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...
8,8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,"3,436,077.00","-34.5598729,-58.443362",-34.56,...,60.00,"3,000.00","3,250.00",nan,nan,nan,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121209,121209,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,"3,436,077.00","-34.563685346,-58.4426825896",-34.56,...,nan,"2,611.46",nan,nan,nan,"6,300.00",http://www.properati.com.ar/1cj7c_venta_depart...,Edificio en esquina de muy buena categoría de ...,Av .Luis Maria Campos 1400- Belgrano-Capital F...,https://thumbs4.properati.com/0/zeLNBuyylOJL7z...
121215,121215,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,"3,436,077.00",NaN,nan,...,93.00,"7,699.12","9,354.84",nan,3.00,"10,000.00",http://www.properati.com.ar/1cja2_venta_depart...,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes,https://thumbs4.properati.com/1/bjms0KnaAnlNoQ...
121217,121217,sell,apartment,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal,"3,433,775.00","-34.5706388726,-58.4755963355",-34.57,...,39.00,"2,858.70","3,371.79",nan,nan,nan,http://www.properati.com.ar/1cja7_venta_depart...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,https://thumbs4.properati.com/9/YAe_-2gRVykADP...
121218,121218,sell,apartment,Plaza Colón,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,nan,NaN,nan,...,48.00,"1,997.92","1,997.92",nan,2.00,nan,http://www.properati.com.ar/1cja8_venta_depart...,"2 Amb al contrafrente, luminoso. El departame...",2 amb. C/ dep. de servicio al contrafrente| Re...,https://thumbs4.properati.com/8/Q12PTvU6BQJ0ib...


In [40]:

patron_floor_0 = re.compile("(?P<tipo>planta baja)",flags = re.IGNORECASE)

patron_floor_1 = re.compile("(?P<tipo>1er piso|primer piso|piso 1)",flags = re.IGNORECASE)
patron_floor_2 = re.compile("(?P<tipo>2do piso|segundo piso|piso 2)",flags = re.IGNORECASE)
patron_floor_3 = re.compile("(?P<tipo>3er piso|tercer piso|piso 3)",flags = re.IGNORECASE)
patron_floor_4 = re.compile("(?P<tipo>4to piso|cuarto piso|piso 4)",flags = re.IGNORECASE)
patron_floor_5 = re.compile("(?P<tipo>5to piso|quinto piso|piso 5)",flags = re.IGNORECASE)
patron_floor_6 = re.compile("(?P<tipo>6to piso|sexto piso|piso 6)",flags = re.IGNORECASE)
patron_floor_7 = re.compile("(?P<tipo>7to piso|séptimo piso|piso 7)",flags = re.IGNORECASE)
patron_floor_8 = re.compile("(?P<tipo>8vo piso|octavo piso|piso 8)",flags = re.IGNORECASE)
patron_floor_9 = re.compile("(?P<tipo>9no piso|noveno piso|piso 9)",flags = re.IGNORECASE)
patron_floor_10 = re.compile("(?P<tipo>10mo piso|décimo piso|piso 10)",flags = re.IGNORECASE)
patron_floor_11 = re.compile("(?P<tipo>11vo piso|onceavo piso|piso 11)",flags = re.IGNORECASE)
patron_floor_12 = re.compile("(?P<tipo>12vo piso|doceavo piso|piso 12)",flags = re.IGNORECASE)
patron_floor_13 = re.compile("(?P<tipo>13vo piso|treceavo piso|piso 13)",flags = re.IGNORECASE)
patron_floor_14 = re.compile("(?P<tipo>14vo piso|cartoceavo piso|piso 14)",flags = re.IGNORECASE)
patron_floor_15 = re.compile("(?P<tipo>15vo piso|quinceavo piso|piso 15)",flags = re.IGNORECASE)

floor_paterns = [patron_floor_0,patron_floor_1,patron_floor_2,patron_floor_3,patron_floor_4,patron_floor_5,
                patron_floor_6,patron_floor_7,patron_floor_8,patron_floor_9,patron_floor_10,patron_floor_11,
                patron_floor_12,patron_floor_13,patron_floor_14,patron_floor_15]

def extractFloor(row):
    if ( row['description'] and isinstance(row['description'],str) ):
        for regex in floor_paterns:
            if (regex.match(row['description'])):
                return floor_paterns.index(regex)
    if ( row['title'] and isinstance(row['title'],str) ):
        for regex in floor_paterns:
            if (regex.match(row['title'])):
                return floor_paterns.index(regex)
    return row['floor']

data.loc[(data.property_type == 'apartment') & (data.floor.isnull()),'floor'] = data.loc[(data.property_type == 'apartment') & (data.floor.isnull())].apply(lambda x: extractFloor(x),axis =1)



In [43]:
data.state_name.value_counts()


Capital Federal                 29018
Bs.As. G.B.A. Zona Norte        24058
Bs.As. G.B.A. Zona Sur          12085
Buenos Aires Costa Atlántica     9059
Córdoba                          9038
Bs.As. G.B.A. Zona Oeste         7333
Santa Fe                         5413
Buenos Aires Interior            1814
Mendoza                           476
Corrientes                        470
Río Negro                         361
Misiones                          348
Neuquén                           324
Tucumán                           213
San Luis                          208
Chubut                            164
Entre Ríos                        130
Salta                             102
La Pampa                           36
Chaco                              36
Tierra Del Fuego                   31
Formosa                            29
Catamarca                          26
Santa Cruz                         19
Jujuy                              10
San Juan                            4
Santiago Del

In [61]:

'''filtramos lugares con 5000 propiedades en adelante '''
data = data.groupby("state_name").filter(lambda x: len(x) > 5000)


In [63]:
''' columna a estrenar '''
patron_a_estrenar = re.compile("(?P<tipo>estrenar)",flags = re.IGNORECASE)

def aestrenar(row):
    if ( row['description'] and isinstance(row['description'],str) ):
        if (patron_a_estrenar.match(row['description'])):
                return True
    if ( row['title'] and isinstance(row['title'],str) ):
        if (patron_a_estrenar.match(row['title'])):
            return True
    return False

data['aAstrenar'] = data.apply(lambda x:aestrenar(x),axis = 1);

 


0         False
1         False
2         False
3         False
4         False
          ...  
121215    False
121216    False
121217    False
121218    False
121219    False
Name: aAstrenar, Length: 96004, dtype: bool

In [64]:
data['aAstrenar'].value_counts()

False    95995
True         9
Name: aAstrenar, dtype: int64

In [68]:
''' columna disposicion del dpto'''

patron_disposicion = re.compile("(?P<tipo>frente|lateral|contrafrente)",flags = re.IGNORECASE)

def disposicion(row):
    disposicion_match = None
    if ( row['description'] and isinstance(row['description'],str) ): 
        disposicion_match = patron_disposicion.search(row['description'])
    if ((patron_disposicion is None) and row['title'] and isinstance(row['title'],str)):
        disposicion_match = patron_disposicion.search(row['title'])
    if (disposicion_match is not None):
        return disposicion_match.groups()[0].lower()
    return None  

data['disposicion'] =  data.apply(lambda x:disposicion(x),axis = 1)



In [86]:
data.loc[data.property_type == 'apartment'].disposicion.isnull().sum()

34174

In [87]:
data.price_aprox_usd.describe()


count       96,004.00
mean       241,128.79
std        385,426.44
min              0.00
25%         90,000.00
50%        145,000.00
75%        270,000.00
max     46,545,445.00
Name: price_aprox_usd, dtype: float64

In [85]:
data.loc[data.price_aprox_usd > 8000000,['state_name','currency','price','price_aprox_usd','property_type','price_usd_per_m2','surface_total_in_m2','title','description']]

,state_name,currency,price,price_aprox_usd,property_type,price_usd_per_m2,surface_total_in_m2,title,description
525,Bs.As. G.B.A. Zona Norte,USD,"11,000,000.00","11,000,000.00",house,"1,466.67","7,500.00","Casa 7500m² con Dependencias en Garay, Juan De...",Total vista al rÃ­o. Sobre 7500 m2 de parque a...
11349,Capital Federal,USD,"8,500,000.00","8,500,000.00",house,"1,085.29","7,832.00",Casa 7832m² con Dependencias en Don Bosco 3400...,Sobre 7832m2 de lote existen 4111m2 cubiertas ...
24548,Capital Federal,USD,"46,545,445.00","46,545,445.00",apartment,232.73,"200,000.00",Departamento venta,dsdffhghvcvbcdfdfghdfhghkjhkhjklhjkfgvghfghnfg...
34638,Bs.As. G.B.A. Zona Norte,USD,"14,000,000.00","14,000,000.00",house,"6,734.01","2,079.00","Casa 2079m² en Del Libertador, Av. 2200, Vicen...",Excelente zona. Cerca de la Quinat Presidencia...
44685,Capital Federal,USD,"8,500,000.00","8,500,000.00",apartment,"20,731.71",410.00,LE PARC ALCORTA - PISO MUY ALTO DE 410 M2 CON ...,CONTACTO: MARTIN PINUS 15- EXCELENTISIMA PROPI...
45689,Santa Fe,USD,"11,111,111.00","11,111,111.00",house,"36,549.71",304.00,VENTA CASA AL FRENTE. 3 DORMITORIOS. 2 BAÑOS. ...,CODIGO: 2062-670 ubicado en: URQUIZA 7900 - P...
55319,Bs.As. G.B.A. Zona Norte,USD,"9,900,000.00","9,900,000.00",house,"9,900.00","1,000.00",Casa importante en lote privilegiado con vista...,Casa importante en lote privilegiado con vista...
57907,Bs.As. G.B.A. Zona Norte,USD,"8,500,000.00","8,500,000.00",house,nan,nan,DON BOSCO 3400,Sobre 7832m2 de lote existen 4111m2 cubiertas ...
67690,Bs.As. G.B.A. Zona Oeste,USD,"9,000,000.00","9,000,000.00",store,nan,nan,Excelente Inmueble Industrial de 13.000 m2,Moreno - CÓDIGO 0167Excelente Inmueble Industr...
69115,Capital Federal,USD,"8,500,000.00","8,500,000.00",store,775.69,"10,958.00",Excelente Nave Industrial de 10.960 m2 sobre t...,C.A.B.A - Villa Soldati - CÓDIGO 0146Excelente...
